In [107]:
import time
import json
import random
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")
seed = 42
random.seed(seed)
np.random.seed(seed)

In [108]:
def load_excel_to_dataframe(file_path):
    """
    Reads an Excel file into a Pandas DataFrame and measures the loading time.
    
    Args:
        file_path (str): The path to the Excel file.

    Returns:
        pd.DataFrame: The loaded DataFrame.
        float: The loading time in seconds.
    """
    start_time = time.time()
    df = pd.read_excel(file_path)
    end_time = time.time()
    loading_time = end_time - start_time
    return df, loading_time

In [109]:
loaded_df, loading_time = load_excel_to_dataframe("20221012_karakterdata_til_studenterprojekt.xlsx")
loaded_df.head(5)

,STUDIENR,UDDANNELSE,KURSKODE,KURSTXT,BEDOMMELSE,SKALA,ECTS,UDPROVNING,CENSUR,BEDOMMELSESDATO
0,STNR000001,"Materiale- og procesteknologi, cand.polyt.",2402,02402 Introduktion til statistik,4,7-trinsskala,5.0,Skriftlig,ekstern censur,2018-12-19
1,STNR000001,"Materiale- og procesteknologi, cand.polyt.",28213,28213 Polymerteknologi,7,7-trinsskala,5.0,Afløsningsopgave,ekstern censur,2019-12-30
2,STNR000001,"Materiale- og procesteknologi, cand.polyt.",41632,41632 Robust design af produkter og mekanismer,7,7-trinsskala,5.0,Afløsningsopgave,intern censur,2019-06-07
3,STNR000001,"Materiale- og procesteknologi, cand.polyt.",41656,41656 Materialer i avancerede anvendelser og p...,10,7-trinsskala,10.0,Skriftlig,ekstern censur,2020-05-27
4,STNR000001,"Materiale- og procesteknologi, cand.polyt.",41661,41661 Metallære,10,7-trinsskala,5.0,Afløsningsopgave,intern censur,2018-12-18


In [110]:
print(loading_time)

24.728745937347412


In [111]:
loaded_df.shape

(327248, 10)

In [112]:
loaded_df.dtypes

STUDIENR                   object
UDDANNELSE                 object
KURSKODE                   object
KURSTXT                    object
BEDOMMELSE                 object
SKALA                      object
ECTS                      float64
UDPROVNING                 object
CENSUR                     object
BEDOMMELSESDATO    datetime64[ns]
dtype: object

### Data Cleaning

In [113]:
loaded_df['BEDOMMELSE'] = pd.to_numeric(loaded_df['BEDOMMELSE'], errors='coerce')
loaded_df['KURSKODE'] = pd.to_numeric(loaded_df['KURSKODE'], errors='coerce').astype('Int64')
loaded_df['BEDOMMELSESDATO'] = pd.to_datetime(loaded_df['BEDOMMELSESDATO'], format='%Y-%m-%d')
loaded_df.dtypes

STUDIENR                   object
UDDANNELSE                 object
KURSKODE                    Int64
KURSTXT                    object
BEDOMMELSE                float64
SKALA                      object
ECTS                      float64
UDPROVNING                 object
CENSUR                     object
BEDOMMELSESDATO    datetime64[ns]
dtype: object

In [114]:
loaded_df.dropna(inplace=True)

In [115]:
loaded_df.shape

(250379, 10)

In [116]:
event_log = loaded_df[['STUDIENR','KURSKODE','KURSTXT','BEDOMMELSE','ECTS', 'BEDOMMELSESDATO']]
event_log.head()

,STUDIENR,KURSKODE,KURSTXT,BEDOMMELSE,ECTS,BEDOMMELSESDATO
0,STNR000001,2402,02402 Introduktion til statistik,4.0,5.0,2018-12-19
1,STNR000001,28213,28213 Polymerteknologi,7.0,5.0,2019-12-30
2,STNR000001,41632,41632 Robust design af produkter og mekanismer,7.0,5.0,2019-06-07
3,STNR000001,41656,41656 Materialer i avancerede anvendelser og p...,10.0,10.0,2020-05-27
4,STNR000001,41661,41661 Metallære,10.0,5.0,2018-12-18


In [117]:
# event_log.to_csv('students_data.csv', index=False)

In [118]:
event_df = event_log.copy()

In [119]:
event_df

,STUDIENR,KURSKODE,KURSTXT,BEDOMMELSE,ECTS,BEDOMMELSESDATO
0,STNR000001,2402,02402 Introduktion til statistik,4.0,5.0,2018-12-19
1,STNR000001,28213,28213 Polymerteknologi,7.0,5.0,2019-12-30
2,STNR000001,41632,41632 Robust design af produkter og mekanismer,7.0,5.0,2019-06-07
3,STNR000001,41656,41656 Materialer i avancerede anvendelser og p...,10.0,10.0,2020-05-27
4,STNR000001,41661,41661 Metallære,10.0,5.0,2018-12-18
...,...,...,...,...,...,...
327241,STNR025846,31598,31598 Engineering World Health Sommerskole i m...,4.0,5.0,2017-07-06
327244,STNR025852,42383,42383 Entrepreneurial Life Skills,4.0,5.0,2021-01-11
327245,STNR025852,42387,42387 Digitale trends for entreprenører,10.0,5.0,2020-12-14
327246,STNR025852,42389,42389 Feasibility studies,10.0,10.0,2021-01-22


In [120]:
event_df["HAS_PREREQUISITE"] = 0
event_df["TOTAL_PREREQUISITE_BEDOMMELSE"] = 0
event_df['KURSKODE'] = event_df['KURSKODE'].astype(str)

In [121]:
prerequisite_courses = {
    "2269":["2105","2158","2141"], # Processing minging prerequisite coureses:
    "2291":["2141","2161","2170"], # System integration prerequisite coureses:
    "2502":["1001","1002","1003","1004","1005","1006","1015","1016","2101","2102","2402","2403","2405","2002","2631","2632","2633","2692"], # Image analysis prerequisite coureses
    "2234":["2232","2233"], # Current Topics in System Security prerequisite coureses:
    "2285":["1017","2101","2105","2180","2156"], # Artificial Intelligence and Multi-Agent Systems prerequisite coureses:
    "2806":["2822","2467"], # Social data analysis and visualization  prerequisite coureses:
    "2582":["2409","2450","27411"], # Computational Data Analysis prerequisite coureses:
    "2516": ["02456"], # Introduction to Deep Learning in Computer Vision prerequisite coureses:
    "34365": ["34315","34338","34346"], # IoT Prototyping prerequisite coureses
    "2110":["2105","2326"] # Algorithms and Data Structures 2 prerequisite coureses
}

In [122]:
# Iterate through the rows and update "TOTAL_PREREQUISITE" based on prerequisites
for index, row in event_df.iterrows():
    kurskode = row["KURSKODE"]
    if kurskode in prerequisite_courses:
        prerequisites = prerequisite_courses[kurskode]
        pr = event_df[(event_df["STUDIENR"] == row["STUDIENR"]) & (event_df["KURSKODE"].isin(prerequisites))]
        event_df.at[index, "HAS_PREREQUISITE"] = len(pr)
        event_df.at[index, "TOTAL_PREREQUISITE_BEDOMMELSE"] = pr['BEDOMMELSE'].sum()

In [123]:
event_df

,STUDIENR,KURSKODE,KURSTXT,BEDOMMELSE,ECTS,BEDOMMELSESDATO,HAS_PREREQUISITE,TOTAL_PREREQUISITE_BEDOMMELSE
0,STNR000001,2402,02402 Introduktion til statistik,4.0,5.0,2018-12-19,0,0
1,STNR000001,28213,28213 Polymerteknologi,7.0,5.0,2019-12-30,0,0
2,STNR000001,41632,41632 Robust design af produkter og mekanismer,7.0,5.0,2019-06-07,0,0
3,STNR000001,41656,41656 Materialer i avancerede anvendelser og p...,10.0,10.0,2020-05-27,0,0
4,STNR000001,41661,41661 Metallære,10.0,5.0,2018-12-18,0,0
...,...,...,...,...,...,...,...,...
327241,STNR025846,31598,31598 Engineering World Health Sommerskole i m...,4.0,5.0,2017-07-06,0,0
327244,STNR025852,42383,42383 Entrepreneurial Life Skills,4.0,5.0,2021-01-11,0,0
327245,STNR025852,42387,42387 Digitale trends for entreprenører,10.0,5.0,2020-12-14,0,0
327246,STNR025852,42389,42389 Feasibility studies,10.0,10.0,2021-01-22,0,0


In [124]:
event_df.dtypes

STUDIENR                                 object
KURSKODE                                 object
KURSTXT                                  object
BEDOMMELSE                              float64
ECTS                                    float64
BEDOMMELSESDATO                  datetime64[ns]
HAS_PREREQUISITE                          int64
TOTAL_PREREQUISITE_BEDOMMELSE             int64
dtype: object

### Build Model

In [125]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [126]:
event_df['Year'] = event_df['BEDOMMELSESDATO'].dt.year
event_df['Month'] = event_df['BEDOMMELSESDATO'].dt.month
event_df = event_df.drop('BEDOMMELSESDATO', axis=1)
event_df

,STUDIENR,KURSKODE,KURSTXT,BEDOMMELSE,ECTS,HAS_PREREQUISITE,TOTAL_PREREQUISITE_BEDOMMELSE,Year,Month
0,STNR000001,2402,02402 Introduktion til statistik,4.0,5.0,0,0,2018,12
1,STNR000001,28213,28213 Polymerteknologi,7.0,5.0,0,0,2019,12
2,STNR000001,41632,41632 Robust design af produkter og mekanismer,7.0,5.0,0,0,2019,6
3,STNR000001,41656,41656 Materialer i avancerede anvendelser og p...,10.0,10.0,0,0,2020,5
4,STNR000001,41661,41661 Metallære,10.0,5.0,0,0,2018,12
...,...,...,...,...,...,...,...,...,...
327241,STNR025846,31598,31598 Engineering World Health Sommerskole i m...,4.0,5.0,0,0,2017,7
327244,STNR025852,42383,42383 Entrepreneurial Life Skills,4.0,5.0,0,0,2021,1
327245,STNR025852,42387,42387 Digitale trends for entreprenører,10.0,5.0,0,0,2020,12
327246,STNR025852,42389,42389 Feasibility studies,10.0,10.0,0,0,2021,1


In [127]:
# Select features and target variable
X = event_df[['KURSKODE', 'ECTS', 'HAS_PREREQUISITE', 'TOTAL_PREREQUISITE_BEDOMMELSE']]
y = event_df['BEDOMMELSE']

In [128]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [129]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

***Linear Regression***

In [130]:
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train)
linear_predictions = linear_model.predict(X_test_scaled)

***Decision Tree Regression***

In [131]:
tree_model = DecisionTreeRegressor()
tree_model.fit(X_train_scaled, y_train)
tree_predictions = tree_model.predict(X_test_scaled)

***Random Forest Regression***

In [132]:
forest_model = RandomForestRegressor()
forest_model.fit(X_train_scaled, y_train)
forest_predictions = forest_model.predict(X_test_scaled)

***Gradient Boosting Regressor***

In [133]:
gboostin_model = GradientBoostingRegressor()
gboostin_model.fit(X_train_scaled, y_train)
gboostin_predictions = gboostin_model.predict(X_test_scaled)

In [135]:
# Evaluate models
def evaluate_model(predictions, y_true):
    mae = mean_absolute_error(y_true, predictions)
    mse = mean_squared_error(y_true, predictions)
    r2 = r2_score(y_true, predictions)
    return mae, mse, r2

linear_metrics = evaluate_model(linear_predictions, y_test)
tree_metrics = evaluate_model(tree_predictions, y_test)
forest_metrics = evaluate_model(forest_predictions, y_test)
gboostin_metrics = evaluate_model(gboostin_predictions, y_test)

print("Linear Regression Metrics:", linear_metrics)
print("Decision Tree Regression Metrics:", tree_metrics)
print("Random Forest Regression Metrics:", forest_metrics)
print("Gradient Boosting Regressor Metrics:", gboostin_metrics)

Linear Regression Metrics: (3.086472218364357, 14.554520970388051, 0.007412629034405205)
Decision Tree Regression Metrics: (2.7414605182823752, 11.465584801939974, 0.2180715051841895)
Random Forest Regression Metrics: (2.7411096932430605, 11.45837853962882, 0.21856295694522443)
Gradient Boosting Regressor Metrics: (2.974815148077186, 13.037268318016032, 0.1108860325474832)


In [142]:
def predict_academic_performance(user_input, model):
    user_df = pd.DataFrame(user_input, index=[0])
    X_user = user_df[['KURSKODE', 'ECTS', 'HAS_PREREQUISITE', 'TOTAL_PREREQUISITE_BEDOMMELSE']]
    prediction = model.predict(X_user)
    return prediction[0]

In [144]:
# Example usage
user_input = {
    'KURSKODE': [12345],
    'ECTS': [5.0],
    'HAS_PREREQUISITE': [1],
    'TOTAL_PREREQUISITE_BEDOMMELSE': [10]
}

In [146]:
l_predicted_grade = predict_academic_performance(user_input, linear_model)
dt_predicted_grade = predict_academic_performance(user_input, tree_model)
rf_predicted_grade = predict_academic_performance(user_input, forest_model)
gb_predicted_grade = predict_academic_performance(user_input, gboostin_model)

In [147]:
print(f'Linear Regression Predicted Grade: {l_predicted_grade}')
print(f'Decision Tree Regression Predicted Grade: {dt_predicted_grade}')
print(f'Random Forest Regression Predicted Grade: {rf_predicted_grade}')
print(f'Gradient Boosting Regressor Predicted Grade: {gb_predicted_grade}')

Linear Regression Predicted Grade: 2880.580330674632
Decision Tree Regression Predicted Grade: 8.358288770053475
Random Forest Regression Predicted Grade: 8.356773419414036
Gradient Boosting Regressor Predicted Grade: 8.653782128998198


***Overall Assessment:***

1. The decision tree, random forest, and gradient boosting models outperform linear regression, capturing more complex relationships in the data.
2. Both random forest and gradient boosting have similar performance, with gradient boosting having a slight edge in terms of R2.
3. Model selection may depend on other factors such as interpretability, computational efficiency, and ease of tuning hyperparameters.